# 🧠 Intelligent Contextual Legal Verification System (ICLVS)
### Demonstrative Runner — Functional Prototype

This notebook illustrates the logical flow of the ICLVS framework:

1. **Document Ingest**  → Receive legal text and metadata.
2. **Contextual Reasoning**  → Detect jurisdiction and clause semantics.
3. **Normative Adaptation**  → Align content with regulatory templates.
4. **Traceability & Audit**  → Generate hash-based verification log.

⚖️ *This demonstrative pipeline is suitable for evidentiary presentation before USCIS under the EB-2 NIW category.*

In [ ]:
import json, re, hashlib, datetime

# === 1. LOAD NORMATIVE DATA ===
with open('normative_rules.json') as f:
    RULES = json.load(f)

jurisdictions = {j['name']: j for j in RULES['jurisdictions']}

print(f"Loaded {len(jurisdictions)} jurisdictions from normative dataset.")

In [ ]:
# === 2. CONTEXTUAL REASONING FUNCTION ===
def verify_clause(text: str, jurisdiction: str) -> dict:
    """Analyzes clause under given jurisdictional rule set."""
    rules = jurisdictions.get(jurisdiction)
    score = 50
    detected = []

    for ctype, cdata in rules['clause_types'].items():
        for term in cdata['must_keywords']:
            if re.search(term, text, re.I):
                score += rules['weights']['must']
                detected.append(term)
        for term in cdata['avoid_keywords']:
            if re.search(term, text, re.I):
                score += rules['weights']['avoid']
                detected.append(f"⚠ {term}")

    score = max(RULES['scoring']['caps']['min'], min(RULES['scoring']['caps']['max'], score))
    risk_band = 'Low' if score >= 75 else 'Medium' if score >= 55 else 'High'
    digest = hashlib.sha256((text + jurisdiction).encode()).hexdigest()
    return {
        'jurisdiction': jurisdiction,
        'risk_score': score,
        'risk_band': risk_band,
        'detected_terms': detected,
        'trace_hash': digest,
        'trace_id': f"ICLVS-{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}",
        'timestamp_iso': datetime.datetime.now().isoformat()
    }

In [ ]:
# === 3. SAMPLE EXECUTION ===
sample_clause = "This Agreement shall be governed by the laws of Florida and venue shall be in Miami-Dade County. Either party may terminate for material breach not cured within 30 days."

result = verify_clause(sample_clause, 'Florida')
result

In [ ]:
# === 4. NORMATIVE ADAPTATION ===
jur = jurisdictions['Florida']
adapted_text = jur['suggestion_block']

print('--- Suggested Adapted Clause ---')
print(adapted_text)
print('\nRisk Level:', result['risk_band'], '| Score:', result['risk_score'])

In [ ]:
# === 5. TRACEABILITY LOG OUTPUT ===
log_entry = {k: result[k] for k in RULES['output']['fields']}

with open('iclvs_trace_log.json', 'w') as f:
    json.dump(log_entry, f, indent=2)

print('Trace log saved → iclvs_trace_log.json')
print(json.dumps(log_entry, indent=2))

### ✅ Demonstration Completed

This notebook produces:
- Clause verification results (jurisdictional risk score)
- Adapted text suggestion from normative dataset
- SHA-256 traceable log (`iclvs_trace_log.json`)

⚖️ Use this output as **functional proof of system logic** and attach it as an evidentiary annex to your *White Paper* or *USCIS NIW submission*.